# Imports and Data File

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from haversine import haversine_vector

In [ ]:
file_path = '../../data/movements.csv'
movements_data = pd.read_csv(file_path, parse_dates=['datetime'])

# Functions

# Distance Analysis

In [ ]:
summary_statistics = pd.DataFrame()
pd.Series(distance_results.apply(lambda x: np.mean(x)))

In [ ]:
distance_results.apply(lambda x: np.nanmedian(x))

In [ ]:
distance_results.apply(lambda x: np.max(x))

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from haversine import haversine_vector

# Load the data
file_path = '../../data/movements.csv'
movements_data = pd.read_csv(file_path, parse_dates=['datetime'])

# Define functions
def get_distance_between_rows(df):
    coords1 = df[['latitude', 'longitude']].values[:-1]
    coords2 = df[['latitude', 'longitude']].values[1:]
    distances = haversine_vector(coords1, coords2)
    return distances

# check
def get_time_between_rows(df):
    date = pd.to_datetime(df['datetime'])
    coords1 = date.values[:-1]
    coords2 = date.values[1:]
    return (coords2 - coords1)

# Calculate distances and times
distance_results = pd.DataFrame()
time_results = pd.DataFrame()
grouped = movements_data.groupby('id')

for name, group in grouped:
    distances = (get_distance_between_rows(group)) # Convert to meters
    times = get_time_between_rows(group)
    distance_results[name] = pd.Series(distances)
    time_results[name] = pd.Series(times)

# Perform element-wise division
velocity_results = distance_results / time_results
velocity_results.columns = [f'Velocity_{col}' for col in velocity_results.columns]

# Display the first 10 rows of the velocity_results DataFrame
print(velocity_results.head(10))

for col in velocity_results.columns:
    plt.figure(figsize=(15, 10))
    plt.plot(velocity_results.index, velocity_results[col], marker='o', linestyle='-', label=col)
    
    # Add labels and title
    plt.xlabel('Index')
    plt.ylabel('Velocity (m/s)')
    plt.title(f'Velocity Over Time for {col}')
    plt.legend()
    
    # Show the plot
    plt.show()

In [ ]:
# Calculate distances and times
distance_results = pd.DataFrame()
time_results = pd.DataFrame()
grouped = movements_data.groupby('id')

for name, group in grouped:
    distances = (get_distance_between_rows(group))
    times = get_time_between_rows(group)
    distance_results[name] = pd.Series(distances)
    time_results[name] = pd.Series(times)

# Perform element-wise division
velocity_results = distance_results / time_results
velocity_results.columns = [f'Velocity_{col}' for col in velocity_results.columns]

# Display the first 10 rows of the velocity_results DataFrame
print(velocity_results.head(10))

# Plot the velocity results for each id in separate graphs with downsampling and rolling averages
for col in velocity_results.columns:
    plt.figure(figsize=(15, 10))
    
    # Downsample by plotting every 1000th point
    downsampled_data = velocity_results[col].iloc[::5000]
    
    # Calculate rolling average with a window of 1000 points
    rolling_avg = velocity_results[col].rolling(window=1000).mean()
    
    plt.plot(downsampled_data.index, downsampled_data, marker='o', linestyle='-', label=f'{col} (Downsampled)')
    plt.plot(rolling_avg.index, rolling_avg, linestyle='-', color='red', label=f'{col} (Rolling Avg)')
    
    # Add labels and title
    plt.xlabel('Index')
    plt.ylabel('Velocity (m/s)')
    plt.title(f'Velocity Over Time for {col}')
    plt.legend()
    
    # Show the plot
    plt.show()

In [ ]:


# Find how often values of less than 1 occur
less_than_one_counts = (velocity_results < 1).sum()

# Display the counts
print(less_than_one_counts)